# Dockermex Project Notebook

## Overview
This notebook provides a comprehensive guide to the Dockermex project - a full-stack webapp for hosting and managing DOOM servers with custom WADs and configurations.

**Version:** 0.3 (Alpha)  
**Primary Source Port:** Odamex  
**Author:** zerofuchs

---

## Table of Contents

1. [Project Goals & Purpose](#goals)
2. [Architecture Overview](#architecture)
3. [Quick Start Guide](#quickstart)
4. [Environment Configuration](#environment)
5. [Docker Components](#docker)
6. [API Documentation](#api)
7. [Configuration Management](#configuration)
8. [Security Features](#security)
9. [Development Workflow](#development)
10. [Troubleshooting](#troubleshooting)
11. [Contributing](#contributing)

---

## 1. Project Goals & Purpose {#goals}

### What is Dockermex?
A full-stack webapp designed for easily hosting DOOM servers with custom WADs and different server configurations, currently focused on Odamex as the source port.

### Target Users
- **Mappers** testing maps/mapsets with groups or co-op scenarios
- **Casual Players** wanting quick server setup for friends/clan mates
- **Community Hosts** running DOOM multiplayer for any purpose

### Design Principles

#### 🎯 **Declarative Webapp**
Users declare what they want; the application handles implementation.

#### 🧩 **Single Purpose Microservices**
- Isolated component failures
- Avoid bundled logic between modules
- Graceful degradation

#### 🚀 **Compose on a Dime**
Docker-based building blocks for testing individual components or full stack.

#### ⚙️ **Clean Configuration Management**
- Nothing hardcoded that matters
- Single source of truth for environments
- Easy integration with secret stores

#### 🔒 **Security First**
- Out-of-box security configuration
- Least privilege access
- Straightforward authentication with solid barriers

#### 👨‍💻 **Developer Friendly**
- Self-documenting code
- Easy local testing
- Simple remote setup
- Accessible for contributors


## 2. Architecture Overview {#architecture}

### High-Level Components

```mermaid
graph TB
    User[👤 User] --> Frontend[🌐 Frontend]
    Frontend --> API[🐍 Python API]
    API --> DB[💾 SQLite DB]
    API --> Docker[🐳 Docker Engine]
    Docker --> Servers[🎮 Odamex Servers]
    
    subgraph "Frontend Layer"
        Frontend
        WAF[🛡️ ModSecurity WAF]
        SSL[🔐 SSL/TLS]
    end
    
    subgraph "Application Layer"
        API
        Auth[🔑 Authentication]
        FileManager[📁 WAD Manager]
    end
    
    subgraph "Data Layer"
        DB
        WADs[📦 WAD Files]
        Configs[⚙️ Server Configs]
    end
```

### Technology Stack

| Component | Technology | Purpose |
|-----------|------------|----------|
| **Frontend** | Nginx + HTML/CSS/JS | Web interface, reverse proxy, SSL termination |
| **API** | Python Flask | Core server logic, container management |
| **Database** | SQLite | User management, WAD registry, metadata |
| **Security** | ModSecurity WAF | Web application firewall |
| **Game Servers** | Odamex (Docker) | Actual DOOM servers |
| **Orchestration** | Docker Compose | Container management and networking |

### Network Architecture

- **Port 443**: HTTPS Frontend (Nginx + ModSecurity)
- **Port 444**: WAD Download Service
- **Port 5000**: Internal API (not exposed)
- **Ports 10666+**: Game server UDP ports


## 3. Quick Start Guide {#quickstart}

### Prerequisites
- Docker & Docker Compose
- Python 3.10+
- Git

### Initial Setup

In [ ]:
# 1. Clone the repository with submodules (API and Frontend)
git clone --recurse-submodules https://github.com/Bladetrain3r/Dockermex.git
cd Dockermex

# 2. Create environment file from template
cp .env.template .env

# 3. Edit .env file with your settings
# Set passwords, paths, and server configuration

# 4. Create required directories
mkdir -p iwads/freeware pwads sqlite service-configs

# 5. Start the web application
docker compose -f docker-compose-webapp-modsec.yml up -d

# 6. Test Odamex servers (optional)
docker compose -f docker-compose-odamex.yml up -d

### First Steps After Setup

1. **Login**: Navigate to `https://your-domain.com` and login with admin credentials
2. **Upload WAD**: Use the upload interface to add a WAD file
3. **Create Config**: Set up a server configuration
4. **Share**: Provide server address to friends
5. **Play**: Prove yourself lord of the frags! 🏆


## 4. Environment Configuration {#environment}

### .env File Structure

The `.env` file is the single source of truth for your Dockermex configuration. Here's a breakdown of the key sections:


In [ ]:
# Security Configuration
SECRET_KEY=your_secret_key_here           # Flask secret key
DB_SALT=your_database_salt               # Database security salt
ADMIN_PASSWORD=your_admin_password       # Initial admin password

# Server Configuration
ODAPORT=10666                           # Default Odamex port
NGINX_VERSION=1.27.5                    # Nginx version for builds

# Resource Limits
ODAMEX_CPU_LIMIT=0.2                    # CPU limit for game servers
ODAMEX_MEMORY_LIMIT=64M                 # Memory limit for game servers
FRONTEND_CPU_LIMIT=1                    # CPU limit for frontend
FRONTEND_MEMORY_LIMIT=256M              # Memory limit for frontend
API_CPU_LIMIT=1                         # CPU limit for API
API_MEMORY_LIMIT=128M                   # Memory limit for API

# File Paths
IWAD_FOLDER=./iwads/freeware            # Location of IWAD files
PWAD_FOLDER=./pwads                     # Location of PWAD files
ODAMEX_CONFIGS_PATH=./configs           # Server configuration files

# WAD Download Configuration
WAD_DOWNLOAD_URL=wads.yourdomain.com    # Your WAD download URL

# SSL Certificates
SSL_PRIVATE_KEY=./.ssl/private.key      # SSL private key
SSL_FULLCHAIN_CERT=./.ssl/fullchain.crt # SSL certificate chain
SSL_WADS_KEY=./.ssl/wads.key           # WAD service SSL key
SSL_WADS_CERT=./.ssl/wads.crt          # WAD service SSL cert

# Frontend Configuration
NGINX_CONF_PATH=./Dockermex_Frontend/nginx/nginx.conf
NGINX_INCLUDES_PATH=./Dockermex_Frontend/nginx/conf.d
MODSECURITY_RULES_PATH=./Dockermex_Frontend/nginx/modsecurity
WEBAPP_ROOT_PATH=./Dockermex_Frontend/html

### Configuration Best Practices

1. **Security**: Use strong, unique passwords and keys
2. **Paths**: Use absolute paths where possible
3. **Resources**: Adjust limits based on your hardware
4. **SSL**: Use valid certificates for production
5. **Backup**: Keep a secure backup of your `.env` file


## 5. Docker Components {#docker}

### Available Dockerfiles

| Dockerfile | Purpose | Build Optimization |
|------------|---------|--------------------|
| `Dockerfile` | Standard Odamex build | Release build |
| `Dockerfile.Alpine` | Lightweight Alpine build | Minimal size |
| `Dockerfile.Freedoom` | With Freedoom WADs included | Freedoom integration |
| `Dockerfile.TinyOdamex` | Minimal size build | MinSizRel build |
| `Dockerfile.Nodamex` | No pre-built WADs | Clean server |
| `Dockerfile.nginx` | ModSecurity + Nginx | Security focused |

### Docker Compose Files

#### Main Compose Files
- **`docker-compose-webapp-modsec.yml`**: Full web application with security
- **`docker-compose-odamex.yml`**: Sample Odamex server configurations  
- **`docker-compose.yml.template`**: Template for custom configurations

### Building Custom Images

In [ ]:
# Build standard Odamex image
docker build -t odamex:latest .

# Build Alpine variant (smaller size)
docker build -f Dockerfiles/Dockerfile.Alpine -t odamex:alpine .

# Build with Freedoom included
docker build -f Dockerfiles/Dockerfile.Freedoom -t odamex:freedoom .

# Build Nginx with ModSecurity
docker build -f Dockerfiles/Dockerfile.nginx -t dockermex:nginx-modsec ./Dockermex_Frontend

### Container Resource Management

All containers use configurable resource limits defined in your `.env` file:

- **Game Servers**: Limited CPU/memory to prevent resource exhaustion
- **Frontend**: Higher limits for handling web traffic
- **API**: Moderate limits for API processing

### Networking

- **Bridge Network**: `wad-net` for internal communication
- **Port Mapping**: Game servers expose UDP ports for clients
- **SSL Termination**: Nginx handles all SSL/TLS


## 6. API Documentation {#api}

### Core API Endpoints

The Flask API provides RESTful endpoints for managing the entire system:

#### Authentication
- `POST /login` - User authentication
- `POST /logout` - Session termination
- `GET /auth-check` - Verify authentication status

#### WAD Management
- `POST /upload` - Upload WAD files
- `GET /wads` - List available WADs
- `DELETE /wads/{id}` - Remove WAD file

#### Server Configuration
- `POST /server-config` - Create server configuration
- `GET /server-config` - List configurations
- `PUT /server-config/{id}` - Update configuration
- `DELETE /server-config/{id}` - Remove configuration

#### Server Management
- `POST /servers/start` - Start game server
- `POST /servers/stop` - Stop game server
- `GET /servers/status` - Get server status

#### Admin Functions
- `GET /admin/users` - User management
- `POST /admin/users` - Create user
- `GET /admin/sessions` - Session management

### API Security Features

- **Rate Limiting**: Prevents API abuse
- **Input Validation**: Sanitizes all inputs
- **Session Management**: Secure cookie handling
- **Role-Based Access**: Admin vs. user permissions
- **CSRF Protection**: Cross-site request forgery prevention


## 7. Configuration Management {#configuration}

### Odamex Server Configurations

Server configurations are stored in the `configs/` directory and define game rules, networking, and gameplay parameters.

#### Available Presets
- **`dm-modern.cfg`**: Modern deathmatch (16 players, no jump/freelook)
- **`coop-modern.cfg`**: Cooperative play (8 players, modern rules)
- **`coop-full-throttle.cfg`**: Fast-paced cooperative
- **`horde-modern.cfg`**: Horde mode gameplay
- **`tdm-modern.cfg`**: Team deathmatch

#### Configuration Template System

Use the provided AWK script to generate templates from existing configs:

In [ ]:
# Generate template from existing config
awk -f generate_config_template.awk configs/dm-modern.cfg > templates/dm-template.cfg

# The template will have placeholders like:
# set sv_hostname "{{SV_HOSTNAME}}"
# set sv_maxplayers "{{SV_MAXPLAYERS}}"
# set sv_fraglimit "{{SV_FRAGLIMIT}}"

### Key Configuration Categories

#### Network Settings
- `sv_hostname`: Server name visible in browser
- `sv_maxplayers`: Maximum player count
- `sv_waddownload`: Enable WAD downloading
- `sv_downloadsites`: WAD download URLs

#### Game Rules
- `sv_gametype`: Game mode (0=Coop, 1=DM, 2=TDM, 4=CTF)
- `sv_fraglimit`: Frag limit for deathmatch
- `sv_timelimit`: Time limit in minutes
- `sv_skill`: Difficulty level (1-5)

#### Gameplay Modifiers
- `sv_allowjump`: Enable/disable jumping
- `sv_freelook`: Enable/disable mouselook
- `sv_weaponstay`: Weapons stay after pickup
- `sv_itemsrespawn`: Items respawn after pickup

### Service Configuration

Service configurations in `service-configs/` define how Docker containers are launched with specific parameters:

```json
{
  "service_name": "doom_coop_server",
  "image": "odamex:latest",
  "ports": {"10666/udp": 10666},
  "environment": {
    "IWAD": "freedoom2.wad",
    "CONFIGFILE": "coop-modern.cfg",
    "ODAPORT": "10666"
  },
  "volumes": {
    "./iwads": {"bind": "/app/iwads", "mode": "ro"},
    "./pwads": {"bind": "/app/pwads", "mode": "ro"},
    "./configs": {"bind": "/app/config", "mode": "ro"}
  }
}
```


## 8. Security Features {#security}

### Web Application Firewall (ModSecurity)

- **OWASP Core Rule Set**: Protection against common attacks
- **Request Filtering**: Blocks malicious requests
- **Rate Limiting**: Prevents DoS attacks
- **Logging**: Comprehensive security event logging

### Authentication & Authorization

- **Session-Based Auth**: Secure cookie management
- **Password Hashing**: SHA256 with salt
- **Role-Based Access**: Admin vs. user permissions
- **Session Expiry**: Automatic session cleanup

### Input Validation

- **File Type Validation**: Only `.wad` files accepted
- **Path Sanitization**: Prevents directory traversal
- **Size Limits**: Configurable upload limits
- **Content Scanning**: Malicious content detection

### Network Security

- **SSL/TLS Encryption**: All traffic encrypted
- **HTTPS Redirect**: Automatic HTTP to HTTPS
- **Secure Headers**: Security-focused HTTP headers
- **Internal Networking**: API not exposed externally

### Container Security

- **Non-Root Users**: Containers run as unprivileged users
- **Resource Limits**: Prevents resource exhaustion
- **Read-Only Mounts**: WAD and config directories mounted read-only
- **Network Isolation**: Containers isolated in custom network


## 9. Development Workflow {#development}

### Local Development Setup

In [ ]:
# Development environment setup
git clone --recurse-submodules https://github.com/Bladetrain3r/Dockermex.git
cd Dockermex

# Create development .env
cp .env.template .env.dev
# Edit .env.dev with development settings

# Start development services
docker compose -f docker-compose-webapp-modsec.yml --env-file .env.dev up -d

# View logs
docker compose logs -f

# Development rebuild
docker compose build --no-cache

### Testing

#### Test Categories
- **`tests/docker/`**: Docker container tests
- **`tests/python/`**: API and backend tests  
- **`tests/web/`**: Frontend functionality tests
- **`tests/shell/`**: Shell script tests
- **`tests/full_stack/`**: End-to-end integration tests

#### Running Tests

In [ ]:
# Run specific test category
cd tests/docker
./run_tests.sh

# Run all tests
cd tests
for test_dir in */; do
    echo "Running tests in $test_dir"
    cd "$test_dir"
    ./run_tests.sh
    cd ..
done

### Code Organization

```
Dockermex/
├── Dockermex_Api/           # Backend API code
│   ├── Python/              # Flask application
│   └── Dockerfile.Python    # API container build
├── Dockermex_Frontend/      # Frontend web interface
│   ├── html/                # Static web files
│   ├── nginx/               # Nginx configuration
│   └── Dockerfile           # Frontend container build
├── configs/                 # Odamex server configurations
├── Dockerfiles/             # Various container builds
├── docs/                    # Documentation
├── tests/                   # Test suites
└── service-configs/         # Service definitions
```

### Git Workflow

1. **Feature Branches**: Create branches for new features
2. **Submodules**: Keep submodules updated
3. **Pull Requests**: Use PRs for code review
4. **Testing**: Ensure tests pass before merging
5. **Documentation**: Update docs with changes


## 10. Troubleshooting {#troubleshooting}

### Common Issues

#### Docker Build Failures
**Problem**: Build fails with "no port specified: /udp<empty>"  
**Solution**: Ensure ODAPORT is defined as ARG in Dockerfile and passed as build argument

```dockerfile
ARG ODAPORT=10666
ENV ODAPORT=${ODAPORT}
EXPOSE ${ODAPORT}/udp
```

#### SSL Certificate Issues
**Problem**: "SSL certificate not found"  
**Solution**: Generate self-signed certificates for development:

In [ ]:
# Generate self-signed SSL certificates
mkdir -p .ssl
openssl req -x509 -nodes -days 365 -newkey rsa:2048 \
    -keyout .ssl/private.key \
    -out .ssl/fullchain.crt \
    -subj "/C=US/ST=State/L=City/O=Org/CN=localhost"

# Copy for WAD service
cp .ssl/private.key .ssl/wads.key
cp .ssl/fullchain.crt .ssl/wads.crt

#### Permission Errors
**Problem**: "Permission denied" errors in containers  
**Solution**: Check file ownership and permissions:

In [ ]:
# Fix permissions for mounted volumes
sudo chown -R 1000:1000 iwads pwads configs
chmod -R 755 iwads pwads configs

#### Server Not Starting
**Problem**: Odamex server fails to start  
**Solution**: Check logs and configuration:

In [ ]:
# Check container logs
docker logs <container_name>

# Check if IWAD exists
ls -la iwads/

# Verify configuration file
cat configs/your-config.cfg

# Test configuration manually
docker run -it --rm odamex:latest /app/server/odasrv -config /app/config/your-config.cfg

### Diagnostic Commands

In [ ]:
# Check container status
docker ps -a

# View resource usage
docker stats

# Check network connectivity
docker network ls
docker network inspect dockermex_wad-net

# Test API endpoints
curl -k https://localhost:443/

# Check ModSecurity logs
docker exec -it dockermex-frontend tail -f /var/log/modsec_audit.log

# View disk usage
du -sh iwads/ pwads/ configs/

### Performance Tuning

#### Resource Optimization
- Adjust CPU/memory limits in `.env` based on hardware
- Use Alpine-based images for smaller footprint
- Monitor container resource usage with `docker stats`

#### Network Optimization
- Use dedicated networks for container communication
- Configure appropriate MTU settings
- Monitor network latency to game servers

#### Storage Optimization
- Use bind mounts for development, volumes for production
- Implement WAD deduplication
- Regular cleanup of unused containers and images


## 11. Contributing {#contributing}

### How to Contribute

1. **Fork the Repository**: Create your own fork on GitHub
2. **Create Feature Branch**: `git checkout -b feature/amazing-feature`
3. **Make Changes**: Implement your feature or fix
4. **Test Thoroughly**: Run all relevant test suites
5. **Update Documentation**: Add or update relevant docs
6. **Submit Pull Request**: Create PR with clear description

### Code Standards

- **Python**: Follow PEP 8 style guidelines
- **Docker**: Use multi-stage builds where appropriate
- **Documentation**: Keep README and docs updated
- **Security**: Follow security best practices
- **Testing**: Include tests for new functionality

### Areas for Contribution

#### High Priority
- Configuration parameterization (#25)
- Architecture documentation (#24)
- SIEM integration (#23)
- Server control improvements (#9)

#### Medium Priority
- WAD file validation enhancements
- Additional source port support
- Performance optimizations
- UI/UX improvements

#### Low Priority
- Additional configuration templates
- Extended API functionality
- Additional security hardening
- Documentation improvements

### Support Channels

- **GitHub Issues**: Bug reports and feature requests
- **Doomworld Forums**: Community discussions (zerofuchs)
- **Pull Requests**: Code contributions and reviews

### License

This project is open source. Please review the LICENSE file for details on usage and distribution.

---

## Conclusion

Dockermex represents a comprehensive solution for DOOM server hosting with modern DevOps practices. The combination of containerization, security-first design, and user-friendly interfaces makes it accessible to both technical and non-technical users.

The project's modular architecture allows for easy customization and extension, while the parameterized configuration system ensures flexibility across different deployment scenarios.

Whether you're a mapper testing new content, a community organizer hosting events, or just someone who wants to play DOOM with friends, Dockermex provides the tools and infrastructure to make it happen with minimal friction.

**Happy fragging!** 🎮💀

---

*Last updated: June 2025*  
*Version: 0.3 (Alpha)*  
*Maintainer: zerofuchs*